In [ ]:
def saccade_detection(data, min_len=5, max_vel=40, max_acc=340):
	s_sac = []
	e_sac = []

	x = data["x"]
	y = data["y"]
	time = data["VideoTime"]

	int_dist = (np.diff(x)**2 + np.diff(y)**2)**0.5
	int_time = np.diff(time)

	vel = int_dist / int_time
	acc = np.diff(vel)

	t0i = 0
	stop = False
	while not stop:
		sac_starts = np.where(np.logical_or(vel[t0i+1:] > max_vel, acc[t0i:] > max_acc))[0]
		if len(sac_starts) > 0:
			t1i = t0i + sac_starts[0] + 1
			if t1i >= len(time)-1:
				t1i = len(time)-2
			t1 = time[t1i]

			s_sac.append([t1])

			sac_ends = np.where(np.logical_and(vel[t1i+1:] < max_vel, acc[t1i:] < max_acc))[0]
			if len(sac_ends) > 0:
				t2i = sac_ends[0] + 1 + t1i + 2
				if t2i >= len(time):
					t2i = len(time)-1
				t2 = time[t2i]
				dur = t2 - t1

				if dur >= min_len:
					e_sac.append([t1, t2, dur, x[t1i], y[t1i], x[t2i], y[t2i]])
				else:
					s_sac.pop(-1)

				t0i = 0 + t2i
			else:
				stop = True
		else:
			stop = True

	return s_sac, e_sac

def fixation_detection(data, max_dist=25, min_dur=50):
	s_fix = []
	e_fix = []

	x = data["x"].to_numpy()
	y = data["y"].to_numpy()
	time = data["VideoTime"].to_numpy()

	si = 0
	fix_start = False
	for i in range(1,len(x)):
		squared_distance = ((x[si]-x[i])**2 + (y[si]-y[i])**2)
		dist = 0.0
		if squared_distance > 0:
			dist = squared_distance**0.5
		if dist <= max_dist and not fix_start:
			si = 0 + i
			fix_start = True
			s_fix.append([time[i]])
		elif dist > max_dist and fix_start:
			fix_start = False
			if time[i-1]-s_fix[-1][0] >= min_dur:
				e_fix.append([s_fix[-1][0], time[i-1], time[i-1]-s_fix[-1][0], x[si], y[si]])
			else:
				s_fix.pop(-1)
			si = 0 + i
		elif not fix_start:
			si += 1
	if len(s_fix) > len(e_fix):
		e_fix.append([s_fix[-1][0], time[len(x)-1], time[len(x)-1]-s_fix[-1][0], x[si], y[si]])
	return s_fix, e_fix

def add_saccades(data, e_sac):
	nb_steps = len(data["VideoTime"])
	vts = data["VideoTime"].to_numpy()

	vts = vts.reshape(nb_steps, 1)

	start_times = np.tile(np.array(e_sac)[:,0], (nb_steps, 1))
	end_times = np.tile(np.array(e_sac)[:,1], (nb_steps, 1))

	data['saccade'] = np.any(np.logical_and(vts >= start_times, vts <= end_times), axis=1)

In [ ]:
# tracking_data = pd.read_csv('data/tracking/VR01_MENU_B.csv')
# tracking_data['x'] = np.cos(tracking_data['gaze coord phi'])*np.sin(tracking_data[" gaze coord theta"])
# tracking_data['y'] = np.sin(tracking_data['gaze coord phi'])*np.sin(tracking_data[" gaze coord theta"])
#
# s_sac, e_sac = saccade_detection(tracking_data, min_len=0, max_vel=0.4, max_acc=3.4)
# add_saccades(tracking_data, e_sac)
#
# saccades_removed = tracking_data[tracking_data["saccade"] == False]
# saccades_removed = saccades_removed[saccades_removed["VideoTime"] > 95]
# saccades_removed = saccades_removed[saccades_removed["VideoTime"] < 215]
# plt.scatter(saccades_removed['x'], saccades_removed['y'], alpha=0.1)
# plt.xlim(-1, 1)
# plt.ylim(-1, 1)
# plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import glob
from IPython import display

overlay_width = 7680//6
overlay_height = 3840//6

canvas_x_1 = (overlay_width*1.35)//3
canvas_x_2 = (overlay_width*1.75)//3

canvas_y_1 = (overlay_height*1.2)//4
canvas_y_2 = (overlay_height*2.4)//4

# plt.figure(figsize=(8, 8))
# plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
# plt.xlim(canvas_x_1, canvas_x_2)
# plt.ylim(canvas_y_1, canvas_y_2)
# plt.savefig("overlay.png", bbox_inches='tight')
# plt.show()

demographics = pd.read_csv(f'data/demographics.csv')
for p_id in demographics["Participant ID"]:
	p_gender = demographics[demographics["Participant ID"] == p_id]["Gender"].item()
	p_group = demographics[demographics["Participant ID"] == p_id]["Group"].item()
	p_menu = "A" if p_group == 1 else "B"
	plot_color = "r" if p_gender == 1 else "g"

	overlay_img = imageio.imread(f"data/screenshot_{p_menu}.PNG")
	overlay_img = cv2.resize(overlay_img, (overlay_width, overlay_height))

	tracking_data = pd.read_csv(f'data/tracking/{p_id}_MENU_{p_menu}.csv')
	tracking_data['x'] = tracking_data['gaze coord phi']/np.pi
	tracking_data['y'] = tracking_data[" gaze coord theta"]/(np.pi/2)
	tracking_data['x'] = (tracking_data['x']+1)*0.5*overlay_width
	tracking_data['y'] = (tracking_data['y']+1)*0.5*overlay_height
	tracking_data = tracking_data[tracking_data["VideoTime"] > 95]
	tracking_data = tracking_data[tracking_data["VideoTime"] < 215]

	# plt.figure(figsize=(8, 8))
	# plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
	# plt.scatter(tracking_data['x'], tracking_data['y'], alpha=0.1)
	# plt.xlim(canvas_x_1, canvas_x_2)
	# plt.ylim(canvas_y_1, canvas_y_2)
	# plt.savefig("overlay_tracking.png", bbox_inches='tight')
	# plt.show()

	s_fix, e_fix = fixation_detection(tracking_data, max_dist=15, min_dur=0.05)

	tracking_fixations = pd.DataFrame({"VideoTime": np.array(e_fix)[:,0], "x": np.array(e_fix)[:,3], "y": np.array(e_fix)[:,4]})
	tracking_fixations = tracking_fixations[tracking_fixations["VideoTime"] > 95]
	tracking_fixations = tracking_fixations[tracking_fixations["VideoTime"] < 215]

	# plt.figure(figsize=(8, 8))
	# plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
	# plt.scatter(tracking_fixations['x'], tracking_fixations['y'], color="b", alpha=0.5)
	# plt.xlim(canvas_x_1, canvas_x_2)
	# plt.ylim(canvas_y_1, canvas_y_2)
	# plt.savefig("overlay_fixations.png", bbox_inches='tight')
	# plt.show()

	plt.figure(figsize=(8, 8))
	plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
	i = 0
	cur_x = tracking_fixations.iloc[i]["x"]
	cur_y = tracking_fixations.iloc[i]["y"]
	for i in range(1, len(tracking_fixations)):
		next_x = tracking_fixations.iloc[i]["x"]
		next_y = tracking_fixations.iloc[i]["y"]
		plt.plot([cur_x, next_x], [cur_y, next_y], marker = 'o', c=plot_color, alpha=0.5)
		cur_x = next_x
		cur_y = next_y
	plt.xlim(canvas_x_1, canvas_x_2)
	plt.ylim(canvas_y_1, canvas_y_2)
	plt.savefig(f"gif/trajectory/{'M' if p_gender == 1 else 'F'}_{p_id}.png", bbox_inches='tight')
	plt.close()
	# plt.show()

	# if not os.path.exists(f"gif/{p_id}/"):
	# 	os.mkdir(f"gif/{p_id}/")
	# anim_iter = 0
	# plt.figure(figsize=(8, 8))
	# plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
	# plt.xlim(canvas_x_1, canvas_x_2)
	# plt.ylim(canvas_y_1, canvas_y_2)
	# i = 0
	# cur_x = tracking_fixations.iloc[i]["x"]
	# cur_y = tracking_fixations.iloc[i]["y"]
	# for i in range(1, len(tracking_fixations)):
	# 	next_x = tracking_fixations.iloc[i]["x"]
	# 	next_y = tracking_fixations.iloc[i]["y"]
	# 	plt.plot([cur_x, next_x], [cur_y, next_y], marker = 'o', c=plot_color, alpha=0.5)
	# 	cur_x = next_x
	# 	cur_y = next_y
	# 	plt.savefig(f"gif/{p_id}/{anim_iter:04d}.png", bbox_inches='tight')
	# 	anim_iter += 1
	# plt.close()
	# plt.show()

	# anim_file = f'gif/{p_id}.gif'
	#
	# frames = []
	# filenames = glob.glob(f'gif/{p_id}/*.png')
	# filenames = sorted(filenames)
	# for i, filename in enumerate(filenames):
	# 	frames.append(imageio.imread(filename))
	# for i in range(10):
	# 	frames.append(imageio.imread(filename))
	#
	# imageio.mimsave(anim_file, frames, 'GIF', fps=8)
	# display.Image(filename=anim_file)